# Data Cleansing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## Data Cleansing

This notebook focuses on improving data quality by handling missing values, outliers, and other data issues.
The goal is to prepare a clean dataset for further analysis and modeling.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


In [ ]:
# Load the dataset

In [ ]:
# Note: Replace 'your_data_file.csv' with the actual file path
df = pd.read_csv('your_data_file.csv')


In [ ]:
# Display basic information about the dataset
print("Dataset shape:", df.shape)
print("\nDataset info:")
df.info()

print("\nFirst 5 rows:")
print(df.head())

print("\nDescriptive statistics:")
print(df.describe())


In [ ]:
# 1. Handling missing values

In [ ]:
# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())


In [ ]:
# The dataset appears to have no missing values based on the provided info

In [ ]:
# But we'll include code for handling missing values as a best practice


In [ ]:
# For numeric columns, we could fill with mean/median

In [ ]:
# For categorical columns, we could fill with mode

In [ ]:
# Example:

In [ ]:
# df['Value'] = df['Value'].fillna(df['Value'].median())

In [ ]:
# df['Industry_name_NZSIOC'] = df['Industry_name_NZSIOC'].fillna(df['Industry_name_NZSIOC'].mode()[0])


In [ ]:
# 2. Fix data type issues

In [ ]:
# Convert 'Value' column from object to numeric

In [ ]:
# First, check if there are any non-numeric values
print("\nSample values from 'Value' column:")
print(df['Value'].head())


In [ ]:
# Convert 'Value' column to numeric

In [ ]:
# Using errors='coerce' to convert any non-numeric values to NaN
df['Value'] = pd.to_numeric(df['Value'], errors='coerce')


In [ ]:
# Check if any NaN values were introduced during conversion
print("\nNaN values in 'Value' after conversion:", df['Value'].isna().sum())


In [ ]:
# If there are NaN values, we can investigate and handle them
if df['Value'].isna().sum() > 0:
    print("Examples of rows with non-numeric 'Value':")
    print(df[df['Value'].isna()].head())
    

In [ ]:
    # Fill NaN values with median or drop rows based on analysis
    df['Value'] = df['Value'].fillna(df['Value'].median())


In [ ]:
# 3. Detecting and handling outliers

In [ ]:
# Calculate Z-scores for the 'Value' column
z_scores = stats.zscore(df['Value'])
abs_z_scores = np.abs(z_scores)
outliers = (abs_z_scores > 3)  # Threshold of 3 standard deviations

print("\nNumber of outliers in 'Value' column:", np.sum(outliers))
print("Percentage of outliers:", (np.sum(outliers) / len(df)) * 100, "%")


In [ ]:
# Visualize the distribution of 'Value' to understand outliers
plt.figure(figsize=(10, 6))
sns.histplot(df['Value'], kde=True)
plt.title('Distribution of Value')
plt.xlabel('Value (in millions)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Visualize outliers with a box plot
plt.figure(figsize=(10, 6))
sns.boxplot(y=df['Value'])
plt.title('Box Plot of Value')
plt.ylabel('Value (in millions)')
plt.show()


In [ ]:
# Handle outliers - we can cap them at a certain percentile

In [ ]:
# This is a common approach for financial data where extreme values might be valid
Q1 = df['Value'].quantile(0.25)
Q3 = df['Value'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print("\nOutlier thresholds for 'Value':")
print(f"Lower bound: {lower_bound}")
print(f"Upper bound: {upper_bound}")


In [ ]:
# Create a copy of the original data before capping outliers
df_original = df.copy()


In [ ]:
# Cap the outliers
df['Value_capped'] = df['Value'].clip(lower=lower_bound, upper=upper_bound)


In [ ]:
# Compare original vs capped values
print("\nOriginal 'Value' statistics:")
print(df['Value'].describe())
print("\nCapped 'Value' statistics:")
print(df['Value_capped'].describe())


In [ ]:
# 4. Removing duplicates

In [ ]:
# Check for duplicate rows
duplicate_count = df.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicate_count}")


In [ ]:
# If duplicates exist, remove them
if duplicate_count > 0:
    df = df.drop_duplicates()
    print(f"Removed {duplicate_count} duplicate rows. New shape: {df.shape}")


In [ ]:
# 5. Standardizing formats

In [ ]:
# Clean up Industry_code_ANZSIC06 - extract just the code part

In [ ]:
# First, let's examine the format
print("\nSample ANZSIC06 codes:")
print(df['Industry_code_ANZSIC06'].head())


In [ ]:
# Extract the main ANZSIC06 code (assuming it's the part before any spaces or special characters)
df['ANZSIC06_clean'] = df['Industry_code_ANZSIC06'].str.extract(r'(ANZSIC06 [A-Za-z0-9\-]+)')

print("\nCleaned ANZSIC06 codes:")
print(df['ANZSIC06_clean'].head())


In [ ]:
# Standardize Year format (ensure it's treated as a year, not just a number)

In [ ]:
# Convert to datetime year
df['Year'] = pd.to_datetime(df['Year'], format='%Y').dt.year


In [ ]:
# Create a new column for the decade
df['Decade'] = (df['Year'] // 10) * 10
print("\nDecade distribution:")
print(df['Decade'].value_counts().sort_index())


In [ ]:
# 6. Additional data quality checks

In [ ]:
# Check for consistency between industry codes and names
industry_code_name_pairs = df[['Industry_code_NZSIOC', 'Industry_name_NZSIOC']].drop_duplicates()
print(f"\nUnique industry code-name pairs: {len(industry_code_name_pairs)}")


In [ ]:
# Check if any industry code has multiple names
code_name_counts = df.groupby('Industry_code_NZSIOC')['Industry_name_NZSIOC'].nunique()
inconsistent_codes = code_name_counts[code_name_counts > 1]

if len(inconsistent_codes) > 0:
    print("\nInconsistent industry codes (multiple names):")
    print(inconsistent_codes)
    

In [ ]:
    # Show examples of inconsistencies
    for code in inconsistent_codes.index[:3]:  # Show first 3 examples
        print(f"\nCode {code} has these different names:")
        print(df[df['Industry_code_NZSIOC'] == code]['Industry_name_NZSIOC'].unique())


In [ ]:
# 7. Final cleaned dataset

In [ ]:
# Select and reorder columns for the final dataset
final_columns = [
    'Year', 'Decade', 'Industry_aggregation_NZSIOC', 
    'Industry_code_NZSIOC', 'Industry_name_NZSIOC',
    'ANZSIC06_clean', 'Variable_code', 'Variable_name',
    'Variable_category', 'Value', 'Value_capped', 'Units'
]


In [ ]:
# Create final cleaned dataframe
df_cleaned = df[final_columns]

print("\nFinal cleaned dataset:")
print(df_cleaned.head())
print("\nCleaned dataset info:")
df_cleaned.info()


In [ ]:
# Save the cleaned dataset
df_cleaned.to_csv('nz_industry_financial_data_cleaned.csv', index=False)
print("\nCleaned dataset saved to 'nz_industry_financial_data_cleaned.csv'")


In [ ]:
# Optional: Save the original dataset with outliers identified
df_original['is_outlier'] = outliers
df_original.to_csv('nz_industry_financial_data_with_outliers_flagged.csv', index=False)
print("Original dataset with outliers flagged saved to 'nz_industry_financial_data_with_outliers_flagged.csv'")